# Aggregation

In this notebook we will do the special groupby function that we have defined.

In [2]:
import numpy as np
import pandas as pd

Reading the data and grouping by player.

In [5]:
data = pd.read_csv('Cleaned_Data.csv')
players = data.groupby('player')
players.get_group(4).head()

,Unnamed: 0,player,club,leagueCountry,height,weight,position,games,victories,ties,...,yellowReds,redCards,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,meanRating
1027,1592,4,66,3,190.0,88.0,5,1,1,0,...,0,0,43,0.355498,4078.0,0.000098,0.517225,4238.0,0.000405,0.0
1622,2310,4,66,3,190.0,88.0,5,1,1,0,...,0,0,30,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.0
2400,3136,4,66,3,190.0,88.0,5,2,0,0,...,0,0,30,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.0
4184,5421,4,66,3,190.0,88.0,5,2,0,1,...,0,0,30,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.0
9190,12234,4,66,3,190.0,88.0,5,2,1,0,...,0,0,30,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.0


First we create an empty dataFrame to store the aggregated data and a function to get the most frequenc value in an array.

In [24]:
playerGroups = pd.DataFrame(index = np.arange(0,len(players),1),
                           columns = data.columns)

In [27]:
def most_frequent(arr):
    try:
        counts = np.bincount(arr)
    except:
        return np.mean(arr)
    return np.argmax(counts)

In [56]:
counter = 0
for player in players.groups:
    onePlayer = players.get_group(player)
    
    playerGroups.loc[counter,'club'] = most_frequent(onePlayer.club.values)
    playerGroups.loc[counter,'leagueCountry'] = most_frequent(onePlayer.leagueCountry.values)
    playerGroups.loc[counter,'height'] = most_frequent(onePlayer.height.values)
    playerGroups.loc[counter,'weight'] = most_frequent(onePlayer.weight.values)
    playerGroups.loc[counter,'position'] = most_frequent(onePlayer.position.values)
    
    playerGroups.loc[counter,'games'] = np.sum(onePlayer.games.values)
    playerGroups.loc[counter,'victories'] = np.sum(onePlayer.victories.values)
    playerGroups.loc[counter,'ties'] = np.sum(onePlayer.ties.values)
    playerGroups.loc[counter,'defeats'] = np.sum(onePlayer.defeats.values)
    playerGroups.loc[counter,'goals'] = np.sum(onePlayer.goals.values)
    playerGroups.loc[counter,'yellowCards'] = np.sum(onePlayer.yellowCards.values)
    playerGroups.loc[counter,'yellowReds'] = np.sum(onePlayer.yellowReds.values)
    playerGroups.loc[counter,'redCards'] = np.sum(onePlayer.redCards.values)
    playerGroups.loc[counter,'Alpha_3'] = most_frequent(onePlayer.Alpha_3.values)
    playerGroups.loc[counter,'meanRating'] = most_frequent(onePlayer.meanRating.values)
    
    mean = np.dot(onePlayer.meanIAT.values,onePlayer.nIAT.values)/np.sum(onePlayer.nIAT.values)
    playerGroups.loc[counter,'meanIAT'] = mean
    
    playerGroups.loc[counter,'nIAT'] = np.sum(onePlayer.nIAT.values)
    
    Sum = ((onePlayer.meanIAT.values - mean)**2).sum()
    
    playerGroups.loc[counter,'seIAT'] = np.sqrt(1/np.sum(onePlayer.nIAT.values)*Sum)
    
    meanExp = np.dot(onePlayer.meanExp.values,onePlayer.nExp.values)/sum(onePlayer.nExp.values)
    playerGroups.loc[counter,'meanExp'] = mean
    playerGroups.loc[counter,'nExp'] = np.sum(onePlayer.nExp.values)
    
    StdExp = ((onePlayer.meanExp.values - meanExp)**2).sum()
    playerGroups.loc[counter,'seExp'] = np.sqrt(1/np.sum(onePlayer.nExp.values)*StdExp)
    counter+=1

Now that we have transformed the data we will take a look at it.

In [57]:
playerGroups.head()

,Unnamed: 0,player,club,leagueCountry,height,weight,position,games,victories,ties,...,redCards,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,meanRating,yellowRed
0,NaN,NaN,29,3,172,67,7,364,226,57,...,3,30,0.330002,2.57474e+06,0.000231022,0.330002,2.65594e+06,0.00143119,0.25,1.0
1,NaN,NaN,10,3,183,73,2,294,145,62,...,0,30,0.32911,2.5655e+06,0.00023323,0.32911,2.64746e+06,0.0011793,0.125,0.0
2,NaN,NaN,35,3,171,69,2,178,53,43,...,0,30,0.366673,87911,0.000357719,0.366673,92973,0.00133523,0.25,1.0
3,NaN,NaN,9,3,182,78,2,322,125,82,...,4,30,0.340878,305901,0.000436682,0.340878,321422,0.00261219,0.25,3.0
4,NaN,NaN,66,3,190,88,5,44,22,6,...,1,30,0.348044,105304,0.000458676,0.348044,110781,0.00289355,0,0.0


We drop the player and unamed column because we don't need them.

In [58]:
final = playerGroups.drop(playerGroups.columns[0:2],axis=1)

In [59]:
final.head()

,club,leagueCountry,height,weight,position,games,victories,ties,defeats,goals,...,redCards,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,meanRating,yellowRed
0,29,3,172,67,7,364,226,57,81,32,...,3,30,0.330002,2.57474e+06,0.000231022,0.330002,2.65594e+06,0.00143119,0.25,1.0
1,10,3,183,73,2,294,145,62,87,60,...,0,30,0.32911,2.5655e+06,0.00023323,0.32911,2.64746e+06,0.0011793,0.125,0.0
2,35,3,171,69,2,178,53,43,82,39,...,0,30,0.366673,87911,0.000357719,0.366673,92973,0.00133523,0.25,1.0
3,9,3,182,78,2,322,125,82,115,105,...,4,30,0.340878,305901,0.000436682,0.340878,321422,0.00261219,0.25,3.0
4,66,3,190,88,5,44,22,6,16,0,...,1,30,0.348044,105304,0.000458676,0.348044,110781,0.00289355,0,0.0


And write to CSV.

In [60]:
final.to_csv('playerData.csv')